# Improve the Microscope Pictures with a Camera, a NeoPixel Illuminator and Fourier Ptychography

by [Jose David Marroquin Toledo](https://github.com/josemarroquintoledo/)

## 1 Import Functions from Other Jupyter Notebooks

In [ ]:
%run ../jupyter-modules/fwdimaging.ipynb
%run ../jupyter-modules/phaseretrieval.ipynb
%run ../jupyter-modules/microscopx.ipynb
%run ../jupyter-modules/fputilities.ipynb

## 2 Takes the Photos

In [ ]:
N_PHOTOS = 47
CAM = 1  # Run ls -ltrh /dev/video* to list the video devices picked up
         # by the Linux kernel.

mode = 'RGB'
project_path = take_photos(CAM, N_PHOTOS, color=mode)

## 3 Generate the Incident Wave Vectors for the NeoPixel Lamp

In [ ]:
import matplotlib.pyplot as plt

SAMPLE_DISTANCE = 60  # Distance in mm between the NeoPixel
                      # illuminator and the sample.
# {<Pixel configuration>: <angle in degrees measured from the x-axis
#                  for the first Pixel>, ... }
PIXEL_LAMP = {7: -30, 16: 0, 24: 0}

xy_lst_per_ring = get_pixels_xy(rings=PIXEL_LAMP)
xy_lst_lst = sorted(xy_lst_per_ring.values())
xy_lst = list()
for lst in xy_lst_lst:
    # Concatenate all lists of xy_lst_lst.
    xy_lst.extend(lst)
# Generate the incident wave vectors.
wv_xy = gen_wave_vectors(xy_lst, SAMPLE_DISTANCE)
wx, wy = zip(*wv_xy)  # Split the list of xy-coordinates in a tuple
                      # for all x-component values and another for
                      # the y-component.
# ... and plot them.
plt.plot(wx, wy, 'ro')
plt.axis('scaled')
plt.show()

## 4 Import the Lo-Res Image Set

In [ ]:
PREFIX = 'IMG_' + mode.upper()  # Common prefix for all Lo-Res images.

lores_set = import_lores_img_set(project_path, N_PHOTOS,
                               img_prefix=PREFIX)
hires_w_px = lores_set.shape[2] * 4  # Width of the reconstruction in px.
hires_h_px = lores_set.shape[1] * 4  # Height of the reconstruction in
                                     # px.

## 5 Set a Reconstruction Sequence

In [ ]:
# Generate a reconstruction sequence that starts in the center of the
# Fourier spectrum (central Pixel).
seq = list(range(1, N_PHOTOS + 1))

## 6 Reconstruct the Hi-Res Image from the Lo-Res Image Set

In [ ]:
# PX_COLORS_WAVELENT is dictionary that contains the average of the
# wavelength in meters of the Pixel's colors published by the
# manufacturer in: http://www.seeedstudio.com/document/pdf/WS2812B%20Datasheet.pdf
PX_COLORS_WAVELEN = {'r': 625e-9, 'g': 522.5e-9, 'b': 470e-9}
# Set up the parameters for the microscope. REPLACE THEM with your own.
CCD_PX = 3.4e-6  # The size of the employed CCD in meters.
# These two are only to estimate the value of the numerical aperture
# (NA).
FOCUS_DIST = 5e-3  # The focus distance in meters.
PUPIL_DIAMETER = 2.83e-3  # Pupil's aperture in meters.

na = calc_na(FOCUS_DIST, PUPIL_DIAMETER)  # Numerical aperture of the 
                                          # objective lens.
lst_path = list()  # List to store the route of each reconstruction.
for i in range(len(mode)):
    print('<HiResReconstruction-' + mode[i].upper() + '>')
    wavelen = PX_COLORS_WAVELEN[mode.lower()[i]]
    # Generate the coherent transfer function of the coherent imaging
    # system.
    cft, lores_w_px, lores_h_px, dkx, dky, kx, ky = gen_cft(np.array(wv_xy),
        wavelen, CCD_PX, na, hires_w_px, hires_h_px)
    img_set = lores_set[:, :, :, i]
    path = gen_hires_img(img_set, seq, hires_w_px, hires_h_px, cft,
                         lores_w_px, lores_h_px, dkx, dky, kx, ky,
                         project_path, name=('IMG_FP_' +
                                             mode[i].upper()))
    # Add the path of the reconstruction to a list. This follows the
    # same order of mode (string).
    lst_path.append(path)
# Merge the reconstructions per channel in one and save it.
merge_imgs(lst_path, name='IMG_FP_' + mode.upper())